In [ ]:
! pip install keybert
! pip install keybert transformers
# ! pip install kiwipiepy

In [ ]:
! pip install keyphrase-vectorizers

In [1]:
import pandas as pd
from kiwipiepy import Kiwi
from sklearn.feature_extraction.text import CountVectorizer
from transformers import BertModel
from keybert import KeyBERT

c:\Users\MOONSOO\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\MOONSOO\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
c:\Users\MOONSOO\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
c:\Users\MOONSOO\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_

In [2]:
model = BertModel.from_pretrained("skt/kobert-base-v1")
# KeyBERT 모델 초기화 (skt의 Kobert 사용)
kw_model = KeyBERT(model)

# 데이터 Load & Preprocess

In [ ]:
"""문자열 형태의 리스트를 파이썬 리스트로 변환하는 작업. 필요하면 사용"""
import ast  # 문자열 형태의 리스트를 파이썬 리스트로 변환하기 위해 필요

def text_join(row):
    text=row.review_sentence_split
    result=" ".join(text)
    return result

# df['joined_text']=df.apply(text_join,axis=1)
# df.drop(columns=['review_sentence_split'])

# 특정 품사 구만 추출 하는 함수, 목적에 따라 바꿈

In [17]:
from kiwipiepy import Kiwi

kiwi = Kiwi()

# 명사 추출 함수
def noun_extractor(text):
    results = []
    result = kiwi.analyze(text)
    for token, pos, _, _ in result[0][0]:
        if len(token) != 1 and (pos.startswith("N") or pos.startswith("SL")):
            results.append(token)
    return results

#### 부사,조사,어미 제거 함수

In [18]:
# 명사 추출 함수
def adverb_remover(text):
    results = []
    result = kiwi.analyze(text)
    for token, pos, _, len_token in result[0][0]:
        if (
            len_token != 1
            and pos.startswith("J") == False
            and pos.startswith("E") == False
            and pos.startswith("MAJ") == False
        ):
            results.append(token)
    return results

In [100]:
not "JX".startswith("J")

False

In [19]:
# 명사 추출 함수
def extractor(text):
    results = []
    result = kiwi.analyze(text)
    for token, pos, _, _ in result[0][0]:
        if (
            len(token) != 1
            and pos.startswith("N")
            or pos.startswith("SL")
            or pos.startswith("V")
        ):
            results.append(token)
    return results

## KoBERT

In [30]:
model = BertModel.from_pretrained("skt/kobert-base-v1")
# KeyBERT 모델 초기화 (skt의 Kobert 사용)
kw_model = KeyBERT(model)

# 메뉴를 담고있는 리뷰만 추출해서 진행해봄

In [82]:
test = [
    "역시 순대국  먹거리집 역시 맛있어요  둔둔하게 잘먹었습니다  .",
    "근처 순대국중 괜춘   .",
    "구의역 근처 순대국 일등 맛집  여사장님 친절하시고 아쉽게 이제 배민은 안함 머릿고기  순대국 포장 여전히 맛있고 푸짐한데 위생은 조금 더 신경써주시면 좋을 듯.",
    "구의동 순대국 원탑맛집.",
    "오소리 국밥  순대국밥  야채순대 먹었어요",
    "2인분 포장 주문하면 대부분 각 1인분씩 포장 해줘서 따로 말 안 했는데 2인분을 합포장 국물2인분 건더기 2인분  해줘서 1인분씩 포장 아니냐니까 말 안   .",
    "고기모듬 시키면 순대도 수육도 부속고기들도 다 나와서 좋아요",
    "순대국도 맛있고 소주 마시다 식어서 데워달라니 다시 바글바글 끓여주셨어요",
    "와   구의에서 먹어본 순대국중에 1등이네요 ㅎㅎㅎ.",
    "어르신들 사이에 껴서 저도 막걸리에 순대국밥을 적셔봅니당.",
    "순대국  고기모듬  술국 등 모든 메뉴가 맛있어요",
    "순대국 내장국 이일대에서 제일 맛나요   중독돼서 자주 가고 있어요",
    "순대국밥 이랑 막걸리 ㅋㅋ.",
    "순대국도 얼큰하니 진하고 맛있구   모둠고기도 부드럽고 쫄깃쫄깃.",
    "순대국이 맛있어요.",
    "맛집이라 순대국 국물이 진해요",
    "순대국 깔끔.",
    "맛은 있는데 모둠중양이너무 적어서 아쉬웠어요ㅜㅜ.",
    "맛있는 순대국집  가격이 올라 별하나 뺍니다.",
    "순대국밥등 넘 맛있어요",
]

In [83]:
test1=" ".join(test)
test1

'역시 순대국  먹거리집 역시 맛있어요  둔둔하게 잘먹었습니다  . 근처 순대국중 괜춘   . 구의역 근처 순대국 일등 맛집  여사장님 친절하시고 아쉽게 이제 배민은 안함 머릿고기  순대국 포장 여전히 맛있고 푸짐한데 위생은 조금 더 신경써주시면 좋을 듯. 구의동 순대국 원탑맛집. 오소리 국밥  순대국밥  야채순대 먹었어요 2인분 포장 주문하면 대부분 각 1인분씩 포장 해줘서 따로 말 안 했는데 2인분을 합포장 국물2인분 건더기 2인분  해줘서 1인분씩 포장 아니냐니까 말 안   . 고기모듬 시키면 순대도 수육도 부속고기들도 다 나와서 좋아요 순대국도 맛있고 소주 마시다 식어서 데워달라니 다시 바글바글 끓여주셨어요 와   구의에서 먹어본 순대국중에 1등이네요 ㅎㅎㅎ. 어르신들 사이에 껴서 저도 막걸리에 순대국밥을 적셔봅니당. 순대국  고기모듬  술국 등 모든 메뉴가 맛있어요 순대국 내장국 이일대에서 제일 맛나요   중독돼서 자주 가고 있어요 순대국밥 이랑 막걸리 ㅋㅋ. 순대국도 얼큰하니 진하고 맛있구   모둠고기도 부드럽고 쫄깃쫄깃. 순대국이 맛있어요. 맛집이라 순대국 국물이 진해요 순대국 깔끔. 맛은 있는데 모둠중양이너무 적어서 아쉬웠어요ㅜㅜ. 맛있는 순대국집  가격이 올라 별하나 뺍니다. 순대국밥등 넘 맛있어요'

### pos태깅 후 목적에 맞게 특정 구 제거

In [108]:
test2=noun_extractor(test1)

In [ ]:
test2

In [110]:
" ".join(test2)

'순대국 먹거리 맛있 둔하 근처 순대국 구의역 근처 순대국 일등 맛집 사장 친절 아쉽 배민 안함 머릿고기 순대국 포장 맛있 위생 신경 구의동 순대국 맛집 오소리 국밥 순대국 야채 순대 포장 주문 대부분 인분 포장 포장 국물 인분 건더기 인분 포장 고기 모듬 순대 수육 부속 고기 순대국 맛있 소주 구의 순대국 어르신 사이 막걸리 순대국 순대국 고기 모듬 술국 메뉴 맛있 순대국 장국 일대 맛나 중독 순대국 막걸리 순대국 얼큰하 진하 맛있 모둠 고기 부드럽 순대국 맛있 맛집 순대국 국물 진하 순대국 깔끔 모둠 중양 아쉽 맛있 순대국 가격 하나 순대 국밥 맛있'

In [111]:
extract_keywords_from_reviews(" ".join(test2))

Keywords: [('맛나', 0.5996), ('진하', 0.5803), ('술국', 0.5796), ('건더기', 0.578), ('구의역', 0.5679), ('둔하', 0.5554), ('구의', 0.5479), ('근처', 0.5394), ('안함', 0.5315), ('막걸리', 0.5254), ('오소리', 0.4898), ('인분', 0.4797), ('주문', 0.4594), ('순대', 0.4512), ('소주', 0.4288), ('머릿고기', 0.4101), ('하나', 0.4076), ('일대', 0.4056), ('얼큰하', 0.3954), ('순대국', 0.3943)]

--------------------------------------------------------------------------------


# 부사구 제거 후 추출

In [101]:
test1

'역시 순대국  먹거리집 역시 맛있어요  둔둔하게 잘먹었습니다  . 근처 순대국중 괜춘   . 구의역 근처 순대국 일등 맛집  여사장님 친절하시고 아쉽게 이제 배민은 안함 머릿고기  순대국 포장 여전히 맛있고 푸짐한데 위생은 조금 더 신경써주시면 좋을 듯. 구의동 순대국 원탑맛집. 오소리 국밥  순대국밥  야채순대 먹었어요 2인분 포장 주문하면 대부분 각 1인분씩 포장 해줘서 따로 말 안 했는데 2인분을 합포장 국물2인분 건더기 2인분  해줘서 1인분씩 포장 아니냐니까 말 안   . 고기모듬 시키면 순대도 수육도 부속고기들도 다 나와서 좋아요 순대국도 맛있고 소주 마시다 식어서 데워달라니 다시 바글바글 끓여주셨어요 와   구의에서 먹어본 순대국중에 1등이네요 ㅎㅎㅎ. 어르신들 사이에 껴서 저도 막걸리에 순대국밥을 적셔봅니당. 순대국  고기모듬  술국 등 모든 메뉴가 맛있어요 순대국 내장국 이일대에서 제일 맛나요   중독돼서 자주 가고 있어요 순대국밥 이랑 막걸리 ㅋㅋ. 순대국도 얼큰하니 진하고 맛있구   모둠고기도 부드럽고 쫄깃쫄깃. 순대국이 맛있어요. 맛집이라 순대국 국물이 진해요 순대국 깔끔. 맛은 있는데 모둠중양이너무 적어서 아쉬웠어요ㅜㅜ. 맛있는 순대국집  가격이 올라 별하나 뺍니다. 순대국밥등 넘 맛있어요'

In [102]:
test3=adverb_remover(test1)
test3

['순대국',
 '먹거리',
 '맛있',
 '둔하',
 '근처',
 '순대국',
 '구의역',
 '근처',
 '순대국',
 '일등',
 '맛집',
 '사장',
 '친절',
 '아쉽',
 '배민',
 '안함',
 '머릿고기',
 '순대국',
 '포장',
 '맛있',
 '푸짐',
 '위생',
 '신경',
 '구의동',
 '순대국',
 '맛집',
 '오소리',
 '국밥',
 '순대국',
 '야채',
 '순대',
 '포장',
 '주문',
 '대부분',
 '인분',
 '포장',
 '포장',
 '국물',
 '인분',
 '건더기',
 '인분',
 '포장',
 '아니',
 '이',
 '고기',
 '모듬',
 '시키',
 '순대',
 '수육',
 '부속',
 '고기',
 '나오',
 '순대국',
 '맛있',
 '소주',
 '마시',
 '데우',
 '끓이',
 '구의',
 '순대국',
 'ㅎㅎㅎ',
 '어르신',
 '사이',
 '막걸리',
 '순대국',
 '적시',
 '순대국',
 '고기',
 '모듬',
 '술국',
 '모든',
 '메뉴',
 '맛있',
 '순대국',
 '장국',
 '일대',
 '맛나',
 '중독',
 '순대국',
 '막걸리',
 'ㅋㅋ',
 '순대국',
 '얼큰하',
 '진하',
 '맛있',
 '모둠',
 '고기',
 '부드럽',
 '순대국',
 '맛있',
 '맛집',
 '순대국',
 '국물',
 '진하',
 '순대국',
 '깔끔',
 '모둠',
 '중양',
 '아쉽',
 'ㅜㅜ',
 '맛있',
 '순대국',
 '가격',
 '하나',
 '순대',
 '국밥',
 '맛있']

In [105]:
test3=" ".join(test3)

In [106]:
extract_keywords_from_reviews(test3)

Keywords: [('ㅎㅎㅎ', 0.7061), ('맛나', 0.6228), ('건더기', 0.621), ('술국', 0.6113), ('진하', 0.6091), ('구의역', 0.6032), ('ㅋㅋ', 0.6025), ('둔하', 0.5927), ('시키', 0.5884), ('구의', 0.5792), ('근처', 0.5778), ('데우', 0.5774), ('안함', 0.574), ('막걸리', 0.5626), ('ㅜㅜ', 0.5498), ('오소리', 0.5268), ('인분', 0.5197), ('주문', 0.4941), ('순대', 0.4845), ('머릿고기', 0.4534)]

--------------------------------------------------------------------------------


# 메뉴 추출한 샘플들로 돌려보자!

In [3]:
data=pd.read_csv('menu_v3_sample_sentence.csv')

In [4]:
data.head()

,rst_name,review_sentence_split,menu_name_split,org_menu_dict,menu_sentence
0,가장맛있는족발 구의역점,['잡내 안나고 맛있어용 구의 먹자골목 바로 앞에서 접 근성이 매우 좋습니당 지나가...,"['', '족발 하 고', '불 족발 새우젓 마늘 쌈장 쌈 야채', '실속 알뜰 보...","[{' 왕족발+막국수)': ['', '왕족발막국수)', ' 왕족발 막국수)', ' ...",['잡내 안나고 맛있어용 구의 먹자골목 바로 앞에서 접 근성이 매우 좋습니당 지나가...
1,강나루 유황오리주물럭 본점,"['생 양념 둘다 강추 ㅎㅎ 가면 무조건 반반이쥬 ㅎㅎ', '늘 갈때마다 맛있어요 ...","['뼈탕', '양념 주물럭', '모듬한마리', '양념주물럭', '오리 주물럭', '...","[{'오리주물럭': ['오리 주물럭', '오리주물럭']}, {'모듬한마리': ['모...","['미나리랑 같이 먹는 오리주물럭 아주 맛있었어요', '고기시키면 뼈탕이 서비스였는..."
2,고공 구의점,"['구의역 소고기 맛집입니다', '토시살 갈비살 버 섯구이 구성의 소고기 세...","['토시 살', '된장찌개 밥', '가브리 살', '소고기 세트', '된장찌개밥',...","[{'고공세트': ['고공세트', '고공 세트']}, {'소고기세트': ['소고기 ...","['토시살 갈비살 버 섯구이 구성의 소고기 세트 주문했어요', '고기도 정말..."
3,고기반햄반김치찌개&김치찜 아차산본점,"['보글보글 김치찌개에 두부 추가하고 라면사리는 필수 로 넣어먹어요', '김치찌개 ...","['왕 계 이 란 말 이', '고기반김치찌개세트', '고기 반 김치찌개 세트', '...","[{'소불고기': ['소 불고기', '소불고기']}, {'고추장제육': ['고추장제...","['김찌는 맵기조절이가능하여 칼칼하게먹기딱좋고 제육은 불향이나서 짭졸하니 좋아요',..."
4,고향집,"['새조개무침 벌교꼬막 대구탕 새조개무침 미쳤음', '맛있어요', '새조개 진...","['물회', '새꼬막양념', '벌교참꼬막', '간 재미', '새 꼬막', '참 꼬막...","[{'쭈꾸미구이': ['쭈꾸미 구이', '쭈꾸미구이']}, {'짱뚱어탕': ['짱뚱...","['벌교참꼬막 38000 2인분 정도 될듯해요', '굴회무침 30 000 간재미무침..."


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   rst_name               40 non-null     object
 1   review_sentence_split  40 non-null     object
 2   menu_name_split        40 non-null     object
 3   org_menu_dict          40 non-null     object
 4   menu_sentence          40 non-null     object
dtypes: object(5)
memory usage: 1.7+ KB


In [6]:
import ast

# 문자열 형태의 리스트를 파이썬 리스트로 변환
data["menu_sentence"] = data["menu_sentence"].apply(ast.literal_eval)
# 변환된 데이터의 첫 번째 요소 확인을 통해 변환 성공 여부 확인

In [8]:
data['menu_sentence'].dtype

dtype('O')

In [7]:
# 문자열 형태의 리스트를 파이썬 리스트로 변환
data["menu_name_split"] = data["menu_name_split"].apply(ast.literal_eval)
# 변환된 데이터의 첫 번째 요소 확인을 통해 변환 성공 여부 확인

KeyPhrases 추출 실행

In [10]:
docs = []

for review_doc in data['menu_sentence']:
    input_txt = ".".join(review_doc)
    docs.append(input_txt)

In [11]:
docs[:3]

['잡내 안나고 맛있어용 구의 먹자골목 바로 앞에서 접 근성이 매우 좋습니당 지나가면서 자주 봤던 곳인데 서비스 메뉴도 알차고 좋네요.매운 거 좋아   ..아이들이랑 가서 반반 족발 먹었어요.지점마다 서비스 내용이 다른가 봅니다.일단 콩나물국 정말 맛있었어요.서비스 5개 중에 막국수와 주먹밥 중에 엄청 고민했는데 아이들 위해서 주먹밥으로 결정했습니다.반찬들이 정갈하고 하나같이 맛이 좋아요.족발은 냄새 하나도 안나고 부드럽습니다.아이들은 기본 족발을 먹고 저는 매운 족발을 상추와 알배추에 싸 먹었어요.정말 맛있게 잘 먹었습니다  ..양도 푸짐하고  곁들여 먹을 수 있는 각종 김치랑 따뜻한 국물에 서비스 메뉴까지 나와서 좋아요..구의동 먹자.골목에서 롱런하는 이유가 있죠.맨날 배달로만 먹다가 매장에서 먹으니 더 꿀맛입니다  윤기가 좔좔 흐르는거 보이시죠.체인이라고 다 같은 체인은 아니잖아요.친절한 사장님 덕에 맛있고 편안하게 잘 먹고왔어요.매장 안 여기저기서 반반족발을 드시던데 진짜 둘 다 맛있어서 놓치고 싶지않아요.한톨도 남기지 않고 순삭하고는 터지는 배를 부여잡고 나왔네요 ㅎㅎ..퇴근하고 현지인은 약속이라도 한듯 이 집으로 집결한다.왜이리 적게 받으시지 싶을정도로 음식에 진심인 집  오랫동안 텃세높은 광진구에서 살아남은 비결이 고스란히 느껴지는 집  1  보쌈 버너 먹는 동안 따뜻하고 촉촉한 고기를 맛 볼 수 있다.2  김치 보쌈 집의 근본인 김치가 무려 10종  김치를 먹기 위해 보쌈을 시킬 정도  3  서비스 따뜻한 콩나물국으로 입맛 돋우고 생굴 된찌 순두부찌개 막국수 주먹밥 중 취향껏 선택 4  푸짐한 양 둘이가서 푸파하려다가 실패하긴 처음 ..광진구족발원탑답게 대존맛임.역시 족발은 가족     넘넘 맛있게 먹었어요.서비스로 선택한 얼큰순두부찌개도 대존맛..이름그대로 가장 맛있는 족발 보쌈 구의최고맛집입니다.믿고 드시면됨 서비스도 많음..포장하러 갔는데 남자 사장님께서 너무 친절하시고 좋았어요.메뉴도 푸짐하고 양도 많고 전화로도 배민처럼 리뷰 이벤트 가능

In [47]:
from sklearn.feature_extraction.text import CountVectorizer
from kiwipiepy import Kiwi

kiwi = Kiwi()

# 토크나이저 함수 
def extract_keywords(text):
    result = kiwi.analyze(text)
    for token, pos, _, _ in result[0][0]:
        if (pos[0] in ['N', 'V']) & (len(token)>1):   # 명사만 추출, 한 글자는 제외 
            yield token
            
# 단어 빈도 추출 모델 
vectorizer = CountVectorizer(ngram_range=(1,1), tokenizer=extract_keywords)  

In [13]:
from keyphrase_vectorizers import KeyphraseCountVectorizer

# vectorizer = KeyphraseCountVectorizer(spacy_pipeline='ko_core_news_sm', pos_pattern='<N.*>')
def extract_keywords(document,top_n=20):
    # 주어진 리뷰들의 문장 리스트에서 각 문장별로 키워드를 추출하여 출력
    keywords = kw_model.extract_keywords(
        document,
        # vectorizer=vectorizer,
        keyphrase_ngram_range=(2,2),  # 단어 n-gram 범위
        stop_words=None,  # 불용어F
        use_maxsum=False,
        use_mmr=True,
        diversity=0.9,  # 다양성
        top_n=top_n,
        highlight=True
        # candidates=candidate
    )  # 상위 n개 키워드
    print(f"Keywords: {keywords}\n")

In [14]:
for doc in docs[:3]:
    # input_text=adverb_remover(doc)
    input_text=" ".join(doc)
    extract_keywords(doc)
    # print(f"Restaurant : {data['rst_name'].iloc[i]}")
    # print(f"Menu list :{menu_candidates} ")
    print("-" * 80)  # 리뷰 사이 구분선

잡내 안나고 맛있어용 구의 먹자골목 앞에서 근성이 매우 좋습니당 지나가면서 자주 봤던 곳인데 서비스 메뉴도 알차고 
좋네요 매운 좋아 아이들이랑 가서 반반 족발 먹었어요 지점마다 서비스 내용이 다른가 봅니다 일단 콩나물국 정말 
맛있었어요 서비스 중에 막국수와 주먹밥 중에 엄청 고민했는데 아이들 위해서 주먹밥으로 결정했습니다 반찬들이 정갈하고
하나같이 맛이 좋아요 족발은 냄새 하나도 안나고 부드럽습니다 아이들은 기본 먹고 저는 매운 족발을 상추와 알배추에 
먹었어요 정말 맛있게 먹었습니다 양도 푸짐하고 곁들여 먹을 있는 각종 김치랑 따뜻한 국물에 서비스 메뉴까지 나와서 
좋아요 구의동 먹자 골목에서 롱런하는 이유가 있죠 맨날 배달로만 먹다가 매장에서 먹으니 꿀맛입니다 윤기가 좔좔 
흐르는거 보이시죠 체인이라고 같은 체인은 아니잖아요 친절한 사장님 덕에 맛있고 편안하게 먹고왔어요 매장 여기저기서 
반반족발을 드시던데 진짜 맛있어서 놓치고 싶지않아요 한톨도 남기지 않고 순삭하고는 터지는 배를 나왔네요 ㅎㅎ 
퇴근하고 현지인은 약속이라도 한듯 집으로 집결한다 왜이리 적게 받으시지 싶을정도로 음식에 진심인 오랫동안 텃세높은 
광진구에서 살아남은 비결이 고스란히 느껴지는 보쌈 버너 먹는 동안 따뜻하고 촉촉한 있다 김치 보쌈 집의 근본인 김치가 
무려 10종 김치를 위해 보쌈을 시킬 정도 서비스 따뜻한 콩나물국으로 입맛 돋우고 생굴 된찌 순두부찌개 막국수 주먹밥 
취향껏 선택 푸짐한 푸파하려다가 실패하긴 처음 광진구족발원탑답게 대존맛임 역시 족발은 가족 넘넘 먹었어요 서비스로 
선택한 얼큰순두부찌개도 대존맛 이름그대로 가장 맛있는 족발 보쌈 구의최고맛집입니다 믿고 드시면됨 서비스도 많음 
포장하러 갔는데 남자 사장님께서 너무 좋았어요 메뉴도 푸짐하고 양도 많고 전화로도 배민처럼 리뷰 이벤트 받고 오자마자
끓이느라고 사진에는 받은 순두부가 빠졌네여 맛있게 먹었습니다 오늘 행복하게 사랑하는 사람들과 맛난 족발을 먹었습니다
반주도 한잔 족발이 쫀득하고 너무 맛있어요 강추드려요 족발은 가족 우와 족발 맛있어요 밑반찬도 맛있구여 가성비 좋아서
자주 찾아요 비오늘날 매운족발에 쏘맥한잔 즐거운 저녁이에요 족발집인데 너무 맛있어요 밑반찬 구성이 진짜 알찬 
편이에요 서비스로 주시는 막국수도 정말 맛있어서 앞으로 자주 방문할 같아요 처음 뵙는 분과 처음 방문 했는데 맛있구요 
대화하기두 좋구요 맛이 좋아 기분 좋았구요 그래서 대화도 되었어요 다시 오고 싶어요 족발이 부드럽고 맛있어요 부모님과
함께 왔는데 맛있게 드셨답니다 친절한 서비스도 아주 좋았습니다 친구가 맛있다고 여기 지점에서 먹어야한다고 해서 
먹어봤는데 확실히 맛있네요 부들부들 쫄깃한게 진짜 맛있어요 사이드 메뉴도 맛있고 가족끼리 와도 친구랑 먹기에도 너무 
좋아요 앞으로는 여기서 먹어야겠어요 너무 맛있고 상추가 진짜 깨끗해요 족발 맛있게 잘먹구갑니당 1인 막국수도 후루룩 
먹었네용 족발만 먹어봤고 이번에 보쌈을 먹었는데 인생보쌈이에요 ㅜㅜㅜ 진짜 맛있고 반찬도 너무 잘나옴 구의역 존맛 
족발집 퇴근 동료들이랑 하기 좋은 족발집 같아요 음식도 맛있고 사장님도 너무 친절하세요 맛잇고 아주 좋습니다 서비스도
좋고 좋아요 고기도 맛있고 기본으로 나오는 반찬들도 맛있어요 음식도 빠르게 나오는거 같아요 종류도 많아서 각자 원하는
술이 달라도 시킬 있어요 그리고 기본으로 서비스 하나가 무조건 나오는게 너무 좋아요 역시 족발은 구의동 가족 집이 
답십리이지만 족발이 생각나면 항상 여기서만 먹습니다 사장님도 직원분도 왕친절하시고 맛은 말모말모 너무 맛있게 잘먹고
왔어요 가족지점중엔 여기가 제일 나은거 같아요 덕분에 데이트 잘했어요 남편이 좋아하는 지점이에요 냄새도 안나고 
부들쫄깃 맛있습니다 족발의신세계를 느껴보세요 우아 생굴이 서비스 사장님이 완전 팍팍 주시네요 맛은 얼마나 좋은지 
살살 녹습니다 매장에서 따뜻하게 드세요 강추강추 첫방문이었는데 친절하셔서 좋았습니다 깔끔하고 맛도 좋았어요 
여러가지 반찬 곁들임 나와서 더더 좋았습니다 족발 생각나면 항상 옵니다 고기 질도 좋고 푸짐해요 맛은 두말할꺼 없어요 
자주 오겠습니다 메인 족발도 사이드 막국수도 맛있어요 주먹밥까지 시켰더니 맛나네요 친철하고 좋아요 음식이 너무 
맛있고 깔끔해요 사람도 많고 회식장소로도 좋은 곳입니다 주문도 빠르고 빨리 나와서 좋앗어요 우선 너무 맛있음 가족이야
믿고 먹는 곳인데 따끈따끈 먹으니까 맛있고 서비스로 주먹밥도 먹었는데 서비스 주는 것까지 갓벽 맛있게 먹었습니다 맛난
족발 맛있게 먹겠습니다 체인점이라 맛은 거의 유사해요 가장맛족 족발이 달콤한 편인데 맛입니다 직원 대부분 친절해요 
사장님은 친절하고 너무 좋았어요 그건 진짜 감사드 립니다 족발도 너무 맛있었고 피해드리고 싶지 않아 근데 자리에서 
시끄럽게 한거도 아니고 오늘은 반반으로 다른체인점 맛있어요 가게에서 곰팡이 냄새나요 야들야들하구 맛있어요 ㅎㅎ 양도
많구요 ㅎㅎ 여기 족발 시키면 막국수도 서비스더라구요 막국수도 엄청 많구 맛있었어요 ㅎㅎ 밑반찬도 맛있어요 ㅎㅎ 
사장님은 친절해서 좋았으나 젊은 남자 굉장히 불친절합니다 본인 전화 손님한테 조용하라는 불쾌한 상황이 있을 있습니다 
조심하세요 반반했는데 불족은 별루였다는 맛있게 먹었습니다 쫄깃쫄깃 족발과 막국수 정갈한 음식으로 가족이 배부르게 
먹었습니다 날치알 주먹밥 서비스로 좋아요 품질 좋은 족발과 탱탱한 막국수가 맛있는 곳이에요 맛있어요 간이 강하지만 
먹었습니다 족발도 맛나지만 부추 무침이 완전 맛있어요 테이블이 많지는 않지만 친절하세요 반찬 셀프인데 직접 가져다 
주세요 음식 맛나요 족발 맛있네요 간이 세지 않아 좋아요 아쉽게도 사진을 못찍었어요ㅠㅠ 만두 짱맛 ㅠㅠ육수도 짱맛 
맛있고 양도많습니다 배달해서 먹었는데 맛있었어요 맛있어서 아이들이 좋아해요 맛은 있고 서비스도 좋은편이에요 자주 
찾는 가게입니다 집에 먹었어요 맛있어요 포장하면 3천원이 오늘껀 살코기가 적당하지 않았네요 음식 맛있고 정말 
친절하세요 음식 나오는 순서가 먼저 순서는 아니었어요 위생에 둔감하신 분이면 만족스럽게 즐기실 있어요 막국수랑 불족 
맛있어오 맛이 좋다 군대 가기 본인이 알바하던 족발을 먹고 싶다네요 양많고 음식맛있어요 맛있는 족발집 족발은 
맛잇엇는데 가게 자체에서 냄새가 낫어요ㅠㅠ 쿰쿰한 냄새가 가게 전체에서 나더라구요 ㅠㅠ 알바생은 친절했어요 
반반족발시켜 먹었는데 국수도 1인 서비시로 주시고 맛나요 족발 먹을 때마다 맛있고 만족해서 족발 먹을때는 항상 
시켜먹는 곳입니다 포장을 자주 하는데 오늘은 점원분이 착각하셨는지 천원을 받으셨네요 족발 대자 4만원 단골이벤트로 
날치알주먹밥을 선택해서 천원 추가해서 41 000원이고 포장할인 000원 하면 38 000원인데 41 000원에서 포장할인을 
적용하고 000원을 추가하시더라구요 이상해서 두번 말씀드렸는데 맞다고 해서 그냥 39 000원 나왔는데 집에와서 다시 
체크해보니 제가 착각한 아닌 같습니다 그리고 먹으려고 세팅하다보니 주먹밥에 지푸라기같은 섞여있더라구요 야채인가 
했는데 아닌 같아서 빼고 먹었습니다 항상 없고 이런 날도 있는 거겠죠 앞으로 체크 잘해주시길 바랍니다 단골 맛집입니다 
맛나게 먹었습니다 족발이 쫄깃쫄깃 맛나요 매운 불닭 맛있어요 전화주문으로 했는데 사장님이 엄청 친절하셨어요 맛있어요
냄새도 없고 맛있어요 매운맛 양념이 너무 달긴했지만요 양많고 넓어요 서비스가 좋아요 너무 맛있고 너무 푸짐해요 대만족
다른 지점은 4시에 족발 가는하다는데 여긴 1시에도 되네요 조금 멀지만 이용하게 된곳 친절하고 빨리 나와서 좋았어요 
냉채족발 굿굿 사이드 메뉴로 순두부찌개 시켰는데 맛있었어용 있어요 깔끔한 족발이었어요 깻잎도 그렇고 밑반찬도 너무 
신선했어요 재방문의사있습니다 맛은 있었지만 부추가 얼었어요ㅠ 거리두기 수칙 근처에서 제일 잘지키는거 같아요 잡내 
별로 없는 족발 족발먹다가 보쌈은 처음인데 남편이 맛있데요 푸짐하고 가장맛있는 족발 맛있는 같아요 맛있어요 근데 어제
배달시켜먹었을땐 양이 적은거 같아 아쉬웠어요 ㅠㅠ 족발맛나요ㅋ특히 매운맛이 맛났어요ㅎㅎ 맛있는 족발이에요 맛있게 
먹었어용 들를께용 가족은 어느점이든 맛난거 같아요 냄새안나고 맛있어요

Keywords: [('맛있었어요 서비스', 0.6356), ('구의 먹자골목', 0.5431), ('아이들은 기본', 0.3359), ('가게입니다 집에', 0.3316), ('먹었습니다 항상', 0.323), ('10종 김치를', 0.2937), ('서비스 5개', 0.2916), ('곰팡이 냄새나요', 0.2867), ('포장하면 3천원이', 0.2843), ('사장님께서 너무', 0.2672), ('반반으로 다른체인점', 0.2644), ('선택 푸짐한', 0.2527), ('가족 넘넘', 0.2416), ('고기를 있다', 0.2209), ('따뜻하고 촉촉한', 0.2087), ('리뷰 이벤트', 0.2047), ('젊은 남자', 0.1734), ('본인 전화', 0.1468), ('터지는 배를', 0.1163), ('39 000원', 0.0965)]

--------------------------------------------------------------------------------


미나리랑 같이 먹는 오리주물럭 아주 맛있었어요 고기시키면 뼈탕이 서비스였는데 들깨향나는 국물이 맛있었습니다 부추랑 
미나리무침 편하게 먹을 있고 반반 가능해서 주물럭 양념 훈제 가능 고기 뼈탕 나와요 6시가 되면 가득참 소주를 부르는 
찌니가좋아하는 오리주물럭 정말 맛있는집이에요 술집같지만 고기만 드시고가는분들도 은근많은 찐맛집 슴슴한듯 향긋하고 
부드러운 킬링메뉴 여기에 간간하게 바로바로 무쳐내주는 미나리무침 같이 먹으면 푸른초원 활개치는 미친 밸런스 
가족여행으로 오리주물럭 먹을려고 포장했어요 여기 오리주물럭 맛집 미나리무침 달라고 하세요 오리주물럭 비싸지안구 
담백하네요 구의동 오리 주물럭 맛집 맛있어요 찌니가 좋아하는 오리주물럭 좋아요 오리주물럭도 신선하고 미나리무침 
맛있어요 80 이던데 동네 찐맛집인가 솔직 다른집 보다 못느꼈지만 당연히 새만 안난다면 오리주물럭 맛이야 말해뭐해고 
삼채와 미나리와 곁들여먹는 오리주물럭 정말 맛있음 이모들 친절하시고 양도 많고 오리도 반찬도 특히 미나리무침이랑 
뼈탕 서비스 너무 오리고기도 볶음밥도 뼈탕도 짱짱 가성비 괜찮은 오리주물럭집을 찾았습니다 구의동30년토박이예용ㅋ 
맛도맛이지맛 반찬도겁나맛있고 여기 아줌마랑 아저 씨도 친절해요ㅜㅜ 오리주물럭 양념주물 오리주물럭 맛있어요 
오리주물럭이랑 양념 먹었는데 맛있네요 구의역 오리주물럭 매콤한 먹음 ㅋㅋㅋ 뼈탕 맛있어요 양념주물럭 정말 오리주물럭
먹으면 오리뼈탕 무료 주는곳 오랜 단골

Keywords: [('국물이 맛있었습니다', 0.7363), ('가능 고기', 0.6141), ('고기 먹으면', 0.5085), ('좋아하는 오리주물럭', 0.4967), ('맛있어요 80', 0.4521), ('드시고가는분들도 은근많은', 0.4132), ('다른집 보다', 0.374), ('향긋하고 부드러운', 0.3418), ('6시가 되면', 0.3337), ('선택 가능', 0.3136), ('서비스 너무', 0.3013), ('아저 씨도', 0.2727), ('은근많은 동네', 0.2681), ('밸런스 내일', 0.2595), ('푸른초원 활개치는', 0.1543), ('80 어르신들', 0.1481), ('양념주물럭 정말', 0.1382), ('오랜 단골', 0.1032), ('미친 밸런스', 0.0857), ('양념 훈제', 0.0665)]

--------------------------------------------------------------------------------


토시살 갈비살 섯구이 구성의 세트 주문했어요 고기도 정말 맛있었고 사이드로 주문한 쫄면이랑 된장찌개도 맛있었어요 
반찬도 정갈하고 양념도 다양하게 맛보는 재미가 있었고요 특히 구워준다는 점에서 먹기 정말 편했고 고기를 먹고 나와도 
옷에 냄새가 배지 않는 장점이었어요 데이트 장소로나 회식 장소로 좋을 같아요 소고기세트는 토시살이랑 갈비살이 
나오는데 토시살이 대박이에요 소고기 set 주문했는데 토시살 먼저 정성스럽게 구워주시고 다음엔 촉촉한 새송이버섯 
부드러운 갈비살까지 정말 든든하게 먹고 왔어요 마지막으로 식사 메뉴 김치찌개까지 야무지게 먹고 왔어용 김치찌개도 
진하고 고기도 듬뿍이네용 직원분들도 친절하셔서 친구랑도 좋고 가족이랑고 가기 좋아요 구의동에 소고기 맛집 발견했어요
고공에서 소고기세트를 주문해서 먹어봤는데 토시살이 엄청 맛있었습니다 고기를 직접 구워주셔서 편하게 먹을 있고 
사이드메뉴로 된장찌개와 매운쫄면도 맛있었어요 소고기세트로 먹었는데 너무 맛있었고 직접 구워주기까지해서 맛있었어요 
오겹살 진짜 맛도리 기본찬도 정갈하고 김치 하나하나 맛있어서 친구랑 다음에 오자고 했네요 진짜 인생고기집 돼지세트 
시켜먹었는데 진짜 인생 고기 만났어요 목살이 태어나서 먹어본 목살 제일 맛있었어요 사이드메뉴로 매콤쫄면은 주문 필수 
완전 입가심되고 고기랑같이먹기 좋았어요 돼지나 소고기 세트도 합리적인 가격에 종류별로 나오고 직원분이 직접 맛있게 
구워주는것도 한몫합니다 자주 방문 고기집 목살 가브리살 오겹살 먹었습니다 가브리살이 엄청 부드럽고 목살도 너무 
맛있어요 된장찌개도 고기랑 어울리네요 직접 맛있게 구워주시는 구의동맛집 고공 소고기 세트는 토시살과 갈비살 그리고 
버섯구이가 포함되어 있는데 고기 퀄리티 대박이에요 구의동 삼겹살 목살 맛집 고공입니다 둘이 소고기세트 먹으면 양이 
적당해요 반찬 고기 맛있지만 특히 오겹살 너무 맛있었어요 고기 토시살 갈비살 맛있어서 남친이랑 대화도 없이 
흡입해버렸네요 삼겹은 당연하게 너무 맛있었지만 특히 목살이 너무 부드럽고 육즙이 팡팡 터지는게 황홀했습니댜 자주 
방문할게용 돼지 목살과 오겹살도 미쳤습니다 김치찌개 너무 맛있게 먹었어요 돼지세트 구성도 엄청 두꺼워요 토시살이랑 
갈비살 맛있고 밑반찬도 정갈해서 너무 좋았습니다 오늘은 구의역 고기 맛집인 고공 고깃집을 방문했는데용 소고기 세트와 
돼지세트를 먹어서 다양한 부위를 즐기기 위해 시켜보았어요 토시살이 진짜 맛있어요 특히 토시살 살살 녹고 넘나 
맛있었어용 제가 먹어본 목살중 가장 부드러운 목살이었어요 돼지고기 소고기 둘다 너무 아기도 잘먹더라구요 된장찌개까지
맛잇는 고공입니다 여자친구와 방문해서 먹었는데 맛있는 곳은 목살이 부드러운데 여기가 그렇네요 목살 너무 맛있었어요 
국물이 많고 매콤하지 않았습니다 목살만 먹으러 다시 갈거같아요 고공세트 85000 새로 유자하이볼8000 된찌 6000 
버섯추가3000 목살 16000 쫄면 6000 여기 고기 리얼 맛있어요 목살 부드럽고 맛있어요 삼겹살 가브리살 쫄깃 해요 고기 
직접 구워 주셔서 편하게 먹을 있었어요 소고기세트먹었는데 2명이서 먹기좋아요 비빔 고기 오겹살이 엄청 맛있네요 못살과
갈비 먹었는데 갈비 진짜 우오오오오오와 윤기 촤르르 부드러운 꼬소함 버섯이주인공된 초점이지만 눈감아줘유 부끄 저건 
냉면 쫄면 새콤하게 호로록 굽기 귀찮은 날은 고공이다 갈비살 먹어용 맛있었는데 된장찌개는 그냥 그랬어요 직접 
구워주시는 목살 삼겹살 가브리살 너무 맛있어요 동네에 이런 고기맛집이 있을줄이야 자작 자작한 쫄면까지 좋아용 목살이 
맛있어요 특히 목살 목살 육즙이 소고기 세트 시켰는데 맛있어요 양도 푸짐하고 좋네요 토시살맛있어 같이 일행이 먹은 
김치찌개도 김치를 설탕에 재워놨다가 끓인 맛이라고 그르네요 다시 가게 고기만 먹고 사이드는 안시킬거같아요 토시살 
맛있어요 추천 돼지 세트 메뉴 된찌밥 배터져요 직접 구워주시는 고기 평소 목살을 즐겨 먹지않는데 부드럽고 탱탱해서 
너무 맛있었어요 저번에 목살 너무 만족해서 이번엔 토시 먹어 봤는데역시 오늘도 너무 만족 했습니당 사이드메뉴 된장찌개
쫄면도 맛이 좋았고 앞접시를 요청했을 작은 공기와 스푼을 줘서 나눠먹기 편하게 배려받는 느낌이었어요 한우모듬에 
된장찌개 곁들여 식사 했습니다 아이둘 포함 6인이 가서 돼지 세트 2세트씩 먹고 거기에 매콤 쫄면은 진짜 별미별미 오겹살
2인분 추가 마무리했슴다 오늘 말구 저번주에 왔을땐 토시살이 뭔가 평소답지 않게 납작 느낌이라서 조금 의아했는데 맛은 
똑같긴 했구요ㅎㅎ 파김치에 쌀밥 밥말아서 콩나물무침 버섯에 쌈장 기름진 갈비살에 쫄면 그리고 토시살에 와사비 파절임 
츄베릅 여기때문에 다이어트 실패하겠어요ㅠ 다음엔 모임으로 여기서 소주뿌실게요 직원 분들 친절하시고 무엇보다 꼬기 
구워주셔서 대화하기 좋아요 돼지 버섯 된장찌개 쫄면 맛납니다 밑반찬도 정갈하게 나오고 된장찌개도 맛있고 무엇보다 
고기가 넘나 맛있어요 보정 1도없는 영롱한 토시살 입니다 오늘은 정말 힘들게 운동해서 앞머리까지 젖은채로 바로 
토시살을 충전해 주었는데 정말 황홀했습니다 소고기도 부드럽고 맛있지만 돼지 목살도 엄청 부드러워요 특히 목살이 너무 
부드러워서 좋았어요 다음에 올게요 새콤 달콤 매콤한 매력을 있는 쫄면도 너무 맛있었네요 이번에 갔을 사진이 이상하게 
나와서 예전에 찍어둔 사진으로 대체합니당 고기 김치찌개 쫄면 맛있어요 개인적으로는 목살이랑 토시살이 최고ㅠㅠㅠ 특히
토시살은 가성비 흘러내림 소고기도 맛있었는데 목살이 진짜 소고기보다 부드러웠어요 오겹살 맛있습니다 소고기 세트 
먹었는데 고기도 구워주시고 너무 맛있었습니다 소고기도 맛있고 돼지고기도 맛있고 반찬도 맛있고 쫄면도 맛있고 6명이서 
고공세트 2개 소고기세트 1개 쫄면 2개 된장찌개 먹었네여 그리고 돼지고기는 오겹살이라 배부른데도 아주 술술 
들어가더라고요 여긴 냉면 대신 물쫄면이 있어서 고민했는데 생각보다 오겹살이랑 어울리더라고요 고기를 직접 구워주셔서 
너무 편했고 고기랑 된장찌개 뿐만 아니라 기본반찬들까지도 너무 맛있어요 화장실도 깨끗했고 알바생분도 친절하셔서 
서비스도 매우 만족스러웠습니당 옥수수향이 은은하게 목살진짜 맛있습니다 그리고 짱은 유자고추 목살이랑 너무 
잘어울립니당ㅜㅋㅋ 인생목살집 전에 ㄸㅋ보다도 맛있음 정말로 목살 최고 갈비살최고 꼭드샤보세요 고공세트 주문했는데 
가성비 좋네요 고기 맛도 좋아서 목살 추가 주문해서 먹었어요 전체적으로 고기가 좋아보였고 육즙 가득한 그야말로 별미 
중에 별미 김치찌개에 설탕이 많이 넣으셨는지 달았어요 여기 목살이 엄청 부드럽고 옴청맛있어요 ㅠㅠ

Keywords: [('맛있었어요 고기', 0.8065), ('소고기보다 부드러웠어요', 0.5147), ('돼지세트 구성도', 0.5065), ('2세트씩 먹고', 0.4385), ('파김치에 쌀밥', 0.4106), ('세트 메뉴', 0.3978), ('육즙 가득한', 0.3629), ('저건 냉면', 0.3265), ('옥수수향이 은은하게', 0.2993), ('매콤한 매력을', 0.2925), ('토시살은 가성비', 0.2599), ('다시 가게', 0.2366), ('6000 버섯추가3000', 0.2056), ('섯구이 구성의', 0.1924), ('둘다 너무', 0.1855), ('사진으로 대체합니당', 0.1549), ('촉촉한 새송이버섯', 0.1493), ('잘어울립니당ㅜㅋㅋ 인생목살집', 0.141), ('필수 완전', 0.1246), ('85000 새로', 0.0883)]

--------------------------------------------------------------------------------


### 메뉴판 Candidate

In [20]:
def extract_keywords_from_reviews_candy(document,candidate,top_n=20):
    # 주어진 리뷰들의 문장 리스트에서 각 문장별로 키워드를 추출하여 출력
    keywords = kw_model.extract_keywords(
        document,
        keyphrase_ngram_range=(1,2),  # 단어 n-gram 범위
        stop_words=None,  # 불용어F
        use_maxsum=False,
        use_mmr=True,
        diversity=0.9,  # 다양성
        top_n=top_n,
        highlight=True,
        candidates=candidate
    )  # 상위 n개 키워드
    print(f"Keywords: {keywords}\n")

In [22]:
for i in range(40):
    # 식당의 문장 리스트 추출
    review_doc = data["menu_sentence"].iloc[i]
    input_text = ".".join(review_doc)
    menu_candidates=data['menu_name_split'].iloc[i]
    input_text=adverb_remover(input_text)
    input_text=" ".join(input_text)
    extract_keywords_from_reviews_candy(input_text,menu_candidates)
    print(f"Restaurant : {data['rst_name'].iloc[i]}")
    print(f"Menu list :{menu_candidates} ")
    print("-" * 80)  # 리뷰 사이 구분선

잡내 맛있 구의 먹자골목 바로 근성 매우 지나가 자주 서비스 메뉴 알차 아이 반반 족발 지점 서비스 내용 다르 일단 
콩나물국 정말 맛있 서비스 막국수 주먹밥 엄청 고민 아이 위하 주먹밥 결정 반찬 정갈 하나같이 족발 냄새 하나 부드럽 
아이 기본 족발 족발 상추 배추 정말 맛있 푸짐 곁들이 각종 김치 따뜻 국물 서비스 메뉴 나오 구의동 골목 이유 맨날 배달
매장 꿀맛 윤기 좔좔 흐르 보이 체인 체인 아니 친절 사장 맛있 편안 여기저기 반반 족발 진짜 맛있 놓치 남기 순삭 터지 
부여잡 나오 ㅎㅎ 퇴근 현지인 약속 집결 이리 정도 음식 진심 오랫동안 텃세 광진구 살아남 비결 고스란히 느끼 보쌈 버너
동안 따뜻하 촉촉 고기 김치 보쌈 근본 김치 무려 10 김치 위하 보쌈 시키 정도 서비스 따뜻 콩나물국 입맛 돋우 된찌 
순두부찌개 막국수 주먹밥 취향 선택 푸짐 푸파 실패 처음 광진 구족 발원 대조 역시 족발 가족 넘넘 맛있 서비스 선택 
얼큰 순두부찌개 대조 이름 그대로 가장 맛있 족발 보쌈 최고 맛집 서비스 포장 남자 사장 너무 친절 메뉴도 푸짐하 전화 
배민 리뷰 이벤트 가능 끓이 사진 순두부 빠지 맛있 오늘 행복 사랑 사람 맛나 족발 반주 족발 쫀득하 너무 맛있 강추 드리
족발 가족 우와 족발 맛있 밑반찬 맛있 가성비 자주 오늘날 족발 쏘맥 즐겁 저녁 족발 너무 맛있 밑반찬 구성 진짜 알차 
서비스 막국수 정말 맛있 자주 방문 처음 처음 방문 맛있 대화 기분 대화 다시 족발 부드럽 맛있 부모 함께 맛있 드시 친절
서비스 아주 친구 맛있 여기 지점 확실히 맛있 부들부들 쫄깃하 진짜 맛있 사이드 메뉴 맛있 가족 끼리 친구 너무 여기 
너무 맛있 상추 진짜 깨끗하 족발 맛있 막국수 후루룩 족발 이번 보쌈 인생 보쌈 ㅜㅜㅜ 진짜 맛있 반찬 너무 나오 구의역 
족발 퇴근 동료 족발 음식 맛있 사장 너무 친절 아주 서비스 고기 맛있 기본 나오 반찬 맛있 음식 빠르 나오 종류 각자 
원하 시키 기본 서비스 하나 무조건 나오 너무 역시 족발 구의동 가족 답십리 족발 생각나 항상 여기 사장 직원 친절 
말모말모 너무 맛있 가족 지점 여기 제일 덕분 데이트 남편 좋아하 지점 냄새 부들 쫄깃 맛있 족발 세계 느끼 우아 서비스 
사장 완전 팍팍 얼마나 살살 매장 따뜻 강추 강추 방문 친절 깔끔 여러 가지 반찬 곁들이 나오 족발 생각나 항상 고기 푸짐
두말 자주 메인 족발 사이드 막국수 맛있 주먹밥 시키 맛나 음식 너무 맛있 깔끔 사람 회식 장소 주문 빠르 빨리 나오 우선
너무 맛있 가족 따끈따끈 맛있 서비스 주먹밥 서비스 맛있 맛나 족발 맛있 체인점 거의 유사 가장 족발 달콤 직원 대부분 
친절 사장 친절 너무 그거 진짜 감사 족발 너무 맛있 피하 드리 자리 시끄럽 아니 오늘 반반 다른 체인점 맛있 가게 곰팡이
냄새 족발 야들야들 맛있 ㅎㅎ ㅎㅎ 여기 족발 시키 막국수 서비스 막국수 엄청 맛있 ㅎㅎ 밑반찬 맛있 ㅎㅎ 사장 친절 
남자 직원 굉장히 친절 본인 전화 조용 불쾌 상황 조심 반반 별루 맛있 쫄깃쫄깃 족발 막국수 정갈하 음식 가족 배부르 
날치 주먹밥 서비스 품질 족발 탱탱 막국수 맛있 맛있 강하 족발 맛나 부추 무침 완전 맛있 테이블 친절 반찬 셀프 직접 
가지 음식 족발 맛있 사진 ㅠㅠ 만두 ㅠㅠ 육수 맛있 배달 맛있 맛있 아이 서비스 자주 가게 포장 맛있 포장 오늘 살코기 
적당 맛있 정말 친절 음식 나오 순서 먼저 순서 아니 둔감 만족 스럽 즐기 막국수 맛있 군대 본인 알바 족발 음식 맛있 
족발 족발 맛잇 가게 자체 냄새 ㅠㅠ 쿰쿰 냄새 가게 전체 ㅠㅠ 알바 친절 반반 족발 시키 국수 서비시 맛나 족발 맛있 
만족 족발 항상 시키 포장 자주 오늘 점원 착각 족발 대자 단골 이벤트 날치 주먹밥 선택 추가 41 000 포장 할인 000 38 
000 41 000 포장 할인 적용 000 추가 이상 말씀 드리 그냥 39 000 계산 나오 다시 체크 착각 아니 세팅 주먹밥 지푸라기 
섞이 야채 아니 항상 이런 체크 바라 단골 맛집 족발 쫄깃쫄깃 맛나 매우 불닭 맛있 전화 주문 사장 엄청 친절 맛있 냄새 
맛있 매운맛 양념 너무 서비스 너무 맛있 너무 푸짐 만족 다른 지점 족발 여기 조금 이용 친절 빨리 나오 냉채 족발 사이드
메뉴 두부찌개 시키 맛있 깔끔 족발 깻잎 그렇 밑반찬 너무 신선 재방 문의 부추 거리 수칙 근처 지키 잡내 별로 족발 족발
보쌈 처음 남편 맛있 푸짐하 가장 맛있 족발 맛있 맛있 어제 배달 시키 아쉽 ㅠㅠ 족발 맛나 특히 맛나 ㅎㅎ 맛있 족발 
맛있 들르 가족 어느 맛나 냄새 맛있

Keywords: []

Restaurant : 가장맛있는족발 구의역점
Menu list :['', '족발 하 고', '불 족발 새우젓 마늘 쌈장 쌈 야채', '실속 알뜰 보쌈 겉절이', '실속알뜰보쌈겉절이막국수)', '싱글보쌈', '통영어리굴젓알배추', '실속알뜰보쌈 겉절이 막국수)', '족발하고새우젓쌈장마늘쌈야채', '불족발 새우젓 마늘 쌈장 쌈야채 ', '족발 하 고 새우젓 쌈장 마늘 쌈 야채', ' 왕족 발 막국수 )', '실속 알뜰 보쌈', '싱글 족발', '족발 하 고 새우젓 쌈장', ' 왕족 발', '불 족발 새우젓 마늘 쌈장 쌈 야채 ', '족발하고 새우젓 쌈장 마늘 쌈야채 ', '불 족발', '통영 어리굴젓 알배추', ' 왕족발 막국수', '왕족발막국수)', ' 왕족 발 막국수', '싱글족발', ' 왕족발 막국수)', '족발 하 고 새우젓 쌈장 마늘', '싱글 불 족발', '불족발새우젓마늘쌈장쌈야채', '싱글불족발', '수제양념막국수', '통영 어리굴젓 알 배추', '족발 하 고 새우젓 쌈장 마늘 쌈 야채 ', '불 족발 새우젓 마늘 쌈장', '불 족발 새우젓', '실속 알뜰 보쌈 겉절이 막국수 )', '통영 어리굴젓', '왕족발막국수', '싱글 보쌈', '족발 하 고 새우젓', '불 족발 새우젓 마늘', '통영', '수제 양념 막국수'] 
--------------------------------------------------------------------------------


미나리 같이 오리 주물럭 맛있 고기 시키 뼈탕 서비스 들깨 국물 맛있 도리 부추 미나리 무침 편하 반반 가능 주물럭 양념 
훈제 선택 가능 고기 나오 사람 가득 소주 부르 좋아하 오리 주물럭 맛있 술집 고기 은근 동네 슴슴하 향긋 부드럽 오리 
주물럭 메뉴 여기 바로 바로 무치 미나리무침 같이 푸르 초원 활개 오리 미치 밸런스 내일 가족 여행 오리 주물럭 여기 
오리 주물럭 오리 주물럭 담백 구의동 오리 주물럭 맛있 찌니 좋아하 오리 주물럭 오리 주물럭 미나리 무침 맛있 80 어르신
동네 솔직 다른 존맛이네 느끼 당연히 오리 주물럭 곁들이 오리 주물럭 맛있 친절 오리 반찬 특히 미나리무침 서비스 너무 
맛있 오리고기 볶음밥 짱짱 가성비 괜찮 오리 주물럭 30 토박이 반찬 겁나 맛있 여기 아줌마 아저 겁나 친절 ㅜㅜ 오리 
주물럭 주물 오리 주물럭 오리 주물럭 맛있 구의역 오리 주물럭 ㅋㅋㅋ 맛있 양념 주물럭 맛있 오리 주물럭 무료 오랜 단골

Keywords: [('오리 주물럭', 0.4592), ('양념 주물럭', 0.4182), ('뼈탕', 0.163)]

Restaurant : 강나루 유황오리주물럭 본점
Menu list :['뼈탕', '양념 주물럭', '모듬한마리', '양념주물럭', '오리 주물럭', '훈제오리', '모듬 한 마리', '오리주물럭', '뼈 탕', '훈제 오리'] 
--------------------------------------------------------------------------------


토시 갈비 섯구이 구성 소고기 세트 고기 정말 맛있 사이드 주문 쫄면 된장찌개 맛있 반찬 정갈하 양념 다양 맛보 재미 
특히 정말 고기 나오 냄새 장점 데이트 장소 회식 장소 소고기 세트 갈비 나오 토시 대박 소고기 set 주문 토시 먼저 정성 
스럽 다음 촉촉 송이버섯 마지막 부드럽 갈비 정말 든든 마지막 식사 메뉴 김치찌개 야무지 김치찌개 진하 고기 듬뿍 직원 
친절 친구 가족 구의동 소고기 맛집 발견 고공 소고기 세트 토시 엄청 맛있 고기 직접 편하 사이드 메뉴 된장찌개 맛있 
소고기 세트 맛있 직접 맛있 오겹살 진짜 도리 기본 정갈하 김치 하나하나 맛있 친구 다음 진짜 인생 고기집 돼지 세트 
진짜 인생 고기 만나 목살 태어나 목살 맛있 사이드 메뉴 매콤 주문 필수 완전 프레시 입가심 고기 같이 돼지 소고기 세트 
가격 종류 나오 직원 직접 맛있 한몫 자주 방문 고기집 목살 가브리살 오겹살 가브리 엄청 부드럽 목살 너무 맛있 된장찌개
고기 어울리 직접 맛있 구의 맛집 고공 소고기 세트 갈비 버섯 구이 포함 고기 퀄리티 구의동 삼겹살 목살 맛집 고공 둘이 
소고기 세트 반찬 고기 맛있 특히 오겹살 너무 맛있 고기 토시 갈비 맛있 남친 대화 없이 흡입 버리 당연 너무 맛있 특히 
목살 너무 부드럽 육즙 팡팡 터지 황홀했습니댜 자주 방문 돼지 목살 오겹살 미치 김치찌개 너무 맛있 돼지 세트 엄청 두껍
토시 갈비 맛있 밑반찬 정갈하 너무 오늘 의역 고기 고공 고깃집 방문 소고기 세트 세트 다양 부위 즐기 위하 시키 토시 
진짜 맛있 특히 토시 살살 맛있 목살 가장 부드럽 목살 돼지 고기 소고기 너무 부드럽 아기 된장찌개 고공 여자 친구 방문 
목살 부드럽 여기 그렇 목살 너무 맛있 국물 매콤 다시 고공 세트 새로 5000 유자하이볼 8000 된찌 6000 버섯 추가 3000 
공기 1000 목살 16000 쫄면 6000 여기 고기 리얼 맛있 목살 부드럽 맛있 삼겹살 가브리 쫄깃 고기 직접 편하 소고기 세트 
고기 엄청 맛있 갈비 갈비 진짜 우오오오오오 윤기 촤르르 부드럽 꼬소함 버섯 주인공 초점 눈감 부끄 저거 냉면 아니 새콤
호로록 귀찮 고공 갈비 맛있 된장찌개 그냥 그렇 직접 목살 삼겹살 가브리 너무 맛있 동네 이런 고기 자작 자작 쫄면 맛있 
특히 목살 육즙 소고기 세트 맛있 푸짐 토시 살맛 같이 일행 김치찌개 김치 설탕 재우 끓이 그르 다시 고기 사이드 시키 
토시 맛있 추천 돼지 세트 된찌밥 터지 직접 고기 평소 목살 즐기 부드럽 탱탱 너무 맛있 목사 너무 만족 이번 토시 역시 
오늘 너무 만족 사이드 메뉴 된장찌개 쫄면 접시 요청 공기 스푼 나누 편하 배려 느낌 모듬 된장찌개 곁들이 식사 포함 
돼지 세트 거기 매콤 진짜 별미 별미 인분 추가 마무리 오늘 저번 토시 평소 납작 느낌 조금 의아 똑같 ㅎㅎ 파김치 쌀밥 
된찌 콩나물 무침 버섯 쌈장 기름지 갈비 쫄면 토시살에 와사비 절임 츄베릅 여기 때문 다이어트 실패 다음 모임 여기 소주
뿌시 직원 친절 무엇 꼬기 대화 돼지 버섯 된장찌개 맛나 밑반찬 정갈하 나오 된장 찌개 맛있 무엇 고기 맛있 보정 영롱 
토시 오늘 정말 힘들 운동 앞머리 바로 토시 충전 정말 황홀 소고기 부드럽 맛있 돼지 목살 엄청 부드럽 특히 목살 너무 
부드럽 다음 새콤 달콤 매콤 매력 쫄면 너무 맛있 이번 사진 이상 나오 예전 사진 대체 밑반찬 고기 김치찌개 맛있 개인 
목살 토시 최고 ㅠㅠㅠ 특히 토시 가성비 줄줄 흘러내리 소고기 맛있 목살 진짜 소고기 부드럽 목살 맛있 소고기 세트 너무
맛있 소고기 맛있 돼지고기 맛있 반찬 맛있 쫄면 맛있 고공 세트 세트 된장찌개 돼지고기 부르 아주 술술 들어가 여기 냉면
대신 쫄면 고민 생각 어울리 고기 직접 너무 편하 고기 된장찌개 아니 기본 반찬 너무 맛있 화장실 깨끗 알바 친절 서비스 
매우 만족 스럽 옥수수 은은 목살 진짜 맛있 유자 고추 목살 너무 어울리 ㅜㅋㅋ 인생 목살 진심 ㄸㅋ 맛있 정말로 목살 
최고 갈비 최고 꼭드샤보세 고공 세트 가성비 고기 목살 추가 주문 전체 고기 보이 육즙 가득 목살 그야말로 별미 별미 
김치찌개 설탕 많이 여기 엄청 부드럽 맛있 ㅠㅠ

...

c:\Users\MOONSOO\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:1381: UserWarning: Upper case characters found in vocabulary while 'lowercase' is True. These entries will not be matched with any documents
  warnings.warn(


구의 한양대 육회 한우 뭉티기 가장 탱탱 꾸리 쫀득하 식감 미치 스지 전골 메뉴 맛있 육회 육회 진짜 좋아하 이렇게 신선 
육회 사시미 진짜 오랜만 ㅎㅎ 육회 미나리 같이 무치 맛있 사시미 쫀득쫀득 진짜 맛있 구의동 고기 인정 육회 사시미 나오
한우 세트 절임 같이 나오 무순 감태 함께 나오 감태 함께 더욱 꿀맛 사시미 소스 정말 꿀맛 완전 추천 추천 사시미 육회 
접시 나오 한우 세트 분홍 빛깔 신선 사시미 소스 버무리 육회 다른 매력 맛있 쫄깃하 매장 깔끔 한우 한잔 테이블 뭉티 
주문 뭉티기 너무 맛있 육회 신선도 물론 뭉티 처음 전혀 거부감 감태 무순 소스 진짜 환상 초딩 입맛 진짜 존맛탱 존맛탱 
기본 찬인 팔팔 끓이 미역국 같이 너무 맛있 리필 김치 드시 맛있 놀라 메뉴 퇴근 굉장히 지치 맛있 힐링 이렇 엄청 배부르
번창 분위기 음식 신선 특히 녹이 미역국 건더기 솔직히 다른 육회 실패 많이 걱정 걱정 필요 너무 고프 육회 덮밥 시키 
너무 ㅎㅎ 한우 회랑 사시미 너무 부드럽 씹히 육회 소스 너무 맛있 감태 완전 고급 다이닝 기분 너무 맛있 즐기 스지수육 
너무 담백하 풀리 버리 이번 같이 친구 기본 스지 전골 다음 스지 전골 도전 소주 안주 역시 스지 ㅎㅎ 자주 방문 너무 
맛있 구의역 한우 여기 엄청 신선하 미치 레알 쫀득쩐득하니 ㅜㅜ 육회 뭉티기 술안주 딱이에오 추천 모든 소고기 한우 
만드 한양대 육회 다녀오 푸짐 육회 쫄깃 맛있 모두 사시미 그렇 맛있 다음번 메뉴 도전 널리 널리 알리 그런 세트 사시미 
육회 비빔밥 스지 전골 모든 메뉴 하나하나 맛있 한우육회 비빔밥 한우 세트 가득 가득 한우 꾸리 사시미 진짜 너무 맛있 
이것 진짜 고기 느끼 신선 가득 느끼 그것 그렇 주방 스티커 느끼 너무 맛있 식사 메뉴 다양 아주 네이버 예약 가능 편리 
고기 신선하 맛있 미나리 육회 어울리 살살 육회 소주 같이 즐기 구의동 육회 맛집 쫀득하 고소하 그냥 너무 맛있 오뎅 
같이 시키 술안주 너무 맛있 구의역 바로 자주 제대로 사시미 맛집 구의역 한우 사시미 신선하 감태 소스 같이 술안주 최고
특히 성비 불스 전골 스지 푸짐 칼칼 정말 맛있 스지 전골 마지막 국물 볶음밥 방문 의사 200 양대 육회 단골 확정 완전 
신선 너무 맛있 서울 복판 이런 퀄리티 사시미 너무나 행복 같이 시키 불스 지도 완전 맛있 넙적 당면 아니 일반 당면 완전
소주 그냥 순식간 다음 육회 뭉티기 ㅠㅠ 육회 뜨끈하 안주 원하 바로 한양대 육회 고기 진짜 얼마나 신선 고소 스지 전골 
뜨뜬 보신 사골 육수 보신 녹이 맛집 여기 사시미 너무 신선 두께 너무 맛있 스지전골 불스 고민 깔끔 국물 스지 전골 선택
후회 선택 맛있 추가 시키 육회 비빔밥 맛있 다음 스지 방문 여기 소주 진짜 제대로 시작 가볍 소주 스지 전골 시키 진짜 
소주 구의동 육회 한양대 육회 미만 살살 정말 스지 전골 처음 사리 야채 소주 너무 어울리 시원 양념 매콤 달달하 많이 
소면 완전 배부르 너무 만족 스럽 육회 비빔밥 포장 육회 정말 푸짐하 차별 육회 한양대 육회 가세 쫄깃쫄깃 식감 육회 
맛보 쫄깃 육회 향긋 미나리 조화 얼마나 최고 육회 양념 너무 오히려 광진구 육회 여기 정착 ㅎㅅㅎ 시즌 메뉴 매콤 
불스지 전골 매콤하 스트레스 싸악 풀리 부드럽 스지 그냥 사르르르 녹아여 ㅠㅠ 다음 뭉티기도 ㅎㅎ 육회 양념 맛있 
미나리 물리 맛있 진짜 사시미 맛있 완전 추천 김치전 맛있 육회 전문 식당 그렇 자체 근사 신선 없이 요즘 겨울 건대 맛집
권유 드리 단골집 처음 리뷰 영수증 진짜 신선 맛있 ㅎㅎ 비린내 사람 여기 비린내 전혀 뭉티기 감태 짱짱 바라 마음 가득 
스지 전골 구수하 스지에 고기 많이 고추 국물 진하 칼칼 맥주 소주 계속 부르 ㅋㅋㅋ 육회 양념장 미나리 참나물 육회 
초보 고기 당일 고속버스 진짜 신선 영수증 매장 붙이 믿음직스럽 티비 축구 경기 야구 올림픽 월드컵 즐기 뭉티기 구의동 
한양대 육회 다녀오 육향가득에 쫄깃하 대구 출신 찐찐 맛있 같이 나오 감태 와사비 쏘주 한잔 그냥 바로 생각나 당일 도축
한우 뭉티기 생각 들리 술집 꼬들 식감 취향저격 ㅋㅋㅋ 뭉티기 처음 사람 뭉티기 뭉티기 제대로 정말 신선도 그대로 느끼 
상급 품질 한우 뭉티기 맛보 한양대 육회 두툼 찰기 예술 뭉티기 직접 만드 소스 정말 이거 계속 생각 서울 당일 도축 
뭉티기 이것 너무나 만족 스럽 방문 ㅜㅜ 구의동 맛있 육회 딱히 아쉽 양대 진짜 한우 육회 맛보 완전 반하 버리 국내 한우
사용 진짜 살살 일품 ㅠㅠ 한우스지 전골 시키 사골 국물 육수 몸보신 느낌 건강 느낌 사골 국물 야채 당면 호로록 소주 
한잔 정말 생각 ㅎㅎㅎ 기본 안주인 미역국 정말 최고 진짜 맛있 다음 방문 사시미 스지 전골 생각 네이버 예약 웨이팅 
편하 ㅎㅎㅎ 육회 사시미 퀄리티 양푼 육회 비빔밥 나누 언제 항상 너무 신선 오늘 너무 맛있 이렇게 호사 누리 너무 최고 
감태 비싸 많이 고기 서비스 미역국 적당 호로록 스르륵 기분 감태 사시미 올리 말아먹 최고 감태 별로 좋아하 고기 감태 
드시 진짜 도리 환상 가성비 최고 구의동 맛집 한양대 육회 짱짱 추천 드리 회사 진쩌 신선 맛있 쫀득쫀득 식감 한우 맛있 
진짜 육질 스지 전골 육수 진하 맛있 서울 뭉티기 품절 사시미 세트 미치 신선 필요 맛있 너무 기본 미역국 맛있 너무 감태
처음 넉넉 리필 필요 맛있 다음 뭉티기 나오 ㅎㅎ 정말 맛있 자주 만큼 고기 신성 맛있 다음 방문 번창 육회 엄청 비싸 
당일 도축 한우 그렇 신선 쫄깃 부드럽 ㅠㅠㅠ 기본 안주 나오 마카로니 샐러드 미치 구의역 육회 맛집 들어오 너무 신성 
맛있 스지 전골 품절 아쉽 ㅠㅜ 대신 불스 시키 너무 육회 어울리 맛있 사시미 신선하 고기 정말 스지 전골 한우 사골 국물
칼칼 진하 끝내주 육회 신선하 맛있 스지 전골 고기 많이 육수 맛있 술안주 너무 쌀쌀하 겨울철 안주 대구 여기 뭉티기 
대구 김치전 존맛탱 흡입 미나리 미치 맛있 진짜로 다음 다시 만큼 맛있 구의역 맛집 어렵 같이 좋아하 육회러버 아주 여기
당일도축뭉티기를 당일 도축 우둔 그렇 신선도 그냥 미치 버리 빠지 버리 ㅎㅎ 20 시간 뭉티기 식감 변하 당일 판매 제공 
이유 다음 방문 육회 사시미 한우 세트 너무 맛있 밑반찬 맛있 리필 ㅎㅎㅎㅎ 친구 끼리 얘기 너무너무 맛있 소주 부르 
방문 구의동 육회 맛집 사장 친절 음식 너무 맛있 빨리 나오 너무 행복 친절 덕분 맛있 다음 신선 맛있 간단하 마시 육회 
신선 맛있 여러 항상 들어가 ㅎㅎ 육회 유명 많이 구의동 아니 기준 서울 맛있 육회 김치전 전문 보다 맛있 뭉티기 처음 
신선하 친구 음식 너무 맛있 기본 반찬 맛있 ㅎㅎ 맛있 처음 다른 맛있 뭉티기 역시 메인 당일 도축 신선 맛있 소금 새로 
생기 육회 진짜 너무 신선 맛있 최고 ㅎㅎㅎㅎ 엄청 맛도리예 쫀득쫀득 감칠맛 감태 고급 스럽 너무 맛있 ㅎㅎ 사시미 리그
자체 소주 술술 부르 감태 무순 같이 천국 가요 구의역 사시미 추천 너무 맛있 소주 시키 특히 같이 나오 미역국 진짜 맛있
오늘 육회 사시미 다음 뭉티기 육회 진짜 신선 미역국 맛있 완전 꼬숩고 맛있 아래 약간 매콤하 같이 진짜 호록호록 들어가
뭉티기 스지 전골 미나리 여기 뭉티기 절대 살살 진짜 절친 싸우 전골 쏘주 살살 컨디션 인생 뭉티기 영접 쫀득하 맛있 
굉장히 청결 깨끗 너무 ㅎㅎ 육회 러버 완전 강추 기본 안주 플레이팅 너무 예쁘 구성 감태 최고 고기 생각나 친구 오랫만 
대화 시간 보내 맛있 신선 친절 미나리 함께 우리 향긋 담백 부드럽 고소하 미나리 가득 느끼 깔끔 느끼하 술술 들어가 
ㅋㅋㅋ 구의역 추천 예약 덕분 대기 없이 바로 배부르 육회 너무 신선 사르르 가깝 신선 고기 감사 육회 색깔 너무 맛있 
보이 실제로 진짜 맛있 스지 전골 시키 스지 너무 쫄깃하 맛있 세트 사시미 같이 끓이 미역국 맛있 깔끔 밑반찬 훌륭하 
정말 후기 남기 동안 다니 뭉티기 맛집 손가락 정도 진짜 맛있 구의 맛집 구의역 도보 거리 교통 편리 상가 주차장 주차 
가능 육회 뭉치 좋아하 진심 말씀 드리 정말 고기 신선 당연 진짜 예술 이렇 필요 정도 진짜 맛있 마늘 참기름 가득 고추장
양념 진짜 맛있 혼자 뭉티기 너무 맛있 육회 비빔밥 주문 이것 정말 지금 육회 비빔밥

Keywords: [('한우', 0.4899), ('한우 세트', 0.2836)]

Restaurant : 한양대육회집 구의점
Menu list :['', '한우세트', '한우 세트', ' 한우', '배달 B', ' 한우 생차돌 사시미', '배달B무료한우뭉티기', '한우생차돌사시미', '배달 B 무료', '배달 B 무료 한우', '배달 B 무료 한우 뭉티기', ' 한우 생차돌 육회', '배달B 무료 한우 뭉티기 ', ' 한우 생 차돌', '배달 B 무료 한우 뭉티기 ', '한우생차돌육회', ' 한우 생 차돌 사시미', '한우', ' 한우 생 차돌 육회'] 
--------------------------------------------------------------------------------


탕수육 시키 고민 맛있 사람 점심때 몰려들 아침 점심 터미널 휴가 복귀전 마지막 식사 진짜 짬뽕 짜장면 탕수육 처음 
탕수육 부먹 맛있 종업원 중국인 아주머니 계시 너무 친절 본인 무시 화장실 ㅡㅡ 담배 오늘 탕수육 바삭 하도 달달하 맛나
ㅋㅋ 은은 매운맛 짜장 다음 쟁반 짜장 쟁반 짜장 ㅋㅋㅋ 짜장 짬뽕 탕수육 세트 시키 다른 탕슉 넉넉하 너무 ㅠㅠ 짜장면 
황실 짜장 ㅋㅋ 돈까스 짜장 시키 돈까스 기름 너무 아쉽 ㅎㅎㅎㅎ 빠듯 짜장 탕수육 금방 준비 쟁반 짜장 탕수육 세트 
게살 볶음밥 추가 적당 탕수육 쫀득쫀득 맛나 옛날식 짜장면 들어가 감동 양파 최고 고기 잡내 없이 탕수육 바삭 맛있 쟁반
짜장 탕수육 옛날 짜장면 짬뽕 맛보 황실 짜장 추억 세트 시키 탕수육 정말 맛있 쟁반 짜장 탕수육 땅콩 들어가 고소하 
맛있 먼저 사람 아니 단체 음식 먼저 챙기 짜장면 하나 시키 단체 음식 각각 다른 10 먼저 나오 버리 ㅋㅋㅋ 탕수육 진짜 
맛있 황실 세트 짜장 해물 짬뽕 ㅋㅋㅋ 바쁘 사진 맛나 해물 짬뽕 쟁반 짜장

Keywords: [('짜장면', 0.7136), ('쟁반 짜장', 0.5853), ('탕수육', 0.5485), ('해물 짬뽕', 0.4478)]

Restaurant : 황실짜장
Menu list :['삼선간짜장', '해물 볶음 짬뽕', '해물짬뽕', '탕수육', '꽃빵튀김', '쟁반 짜장', '해물 짬뽕', '울 면', '불 짜장', '삼선해물탕', '삼 선 해물탕', '물만두', '불짜장', '물 만 두', '백 짬뽕', '삼 선 간 짜장', '해물볶음짬뽕', '꽃 빵 튀김', '짜장면', '쟁반짜장', '울면', '백짬뽕'] 
--------------------------------------------------------------------------------
